<a href="https://colab.research.google.com/github/siddas18/Natural-Disaster-Damage-Prediction/blob/Kartik/Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Generic Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt
from math import sqrt, isnan, pi, sin, cos, atan2
import requests
import gzip
from functools import reduce
import scipy as scp
import seaborn as sns
import scipy.interpolate

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

# Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

# Metrics
from sklearn.metrics import mean_squared_error, r2_score


import warnings
warnings.catch_warnings()
warnings.simplefilter("ignore")

In [2]:
def get_NOAA_data():
    
    URL = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
    r = requests.get(URL)
    file_names = pd.read_html(r.text)[0]['Name']
    events_file_names = file_names[file_names.str.contains("details",na=False)]
    noaa_list = []
    
    print("Extracting CSV files...")
    for file in events_file_names:
        full_URL = URL + file
        with gzip.open(requests.get(full_URL, stream=True).raw) as f:
            noaa_list.append(pd.read_csv(f))
        
    df = pd.concat(noaa_list)
    
    print("Completed")
    return df

def pickle_source_data():
    noaa_source_df = get_NOAA_data()
    home_dir = os.getcwd()
    data_dir = os.path.join(home_dir, "Data")
    try:
        os.mkdir(data_dir)
        os.chdir(data_dir)
    except OSError:
        os.chdir(data_dir)
        for file in os.listdir():
            os.remove(file)
    noaa_source_df.to_pickle('noaa_source_data.pkl')
    os.chdir(home_dir)
    return noaa_source_df

NOAA_df = pickle_source_data()
    

Extracting CSV files...
Completed


In [3]:
NOAA_df = pd.read_pickle('Data/noaa_source_data.pkl')

In [4]:
def replace_str2num(x):
    if type(x) == float or type(x) == int:
        return float(x)
    num = 1 if x[:-1] == '' else x[:-1]        
    if x[-1] == 'T':
        return float(num) * 1000000000000
    elif x[-1] == 'B':
        return float(num) * 1000000000
    elif x[-1] == 'M':
        return float(num) * 1000000
    elif x[-1] == 'K' or x[-1] == 'k':
        return float(num) * 1000
    elif x[-1] == 'h' or x[-1] == 'H':
        return float(num) * 100
    elif x[-1] == '?':
        return float(num)
    else:
        return float(x)

def winds(x):
    if x['MAGNITUDE_TYPE'] in ['EG', 'E', 'M', 'ES', 'MG', 'MS']:
        return x['MAGNITUDE']

def hail(x):
    if x['MAGNITUDE_TYPE'] not in ['EG', 'E', 'M', 'ES', 'MG', 'MS']:
        return x['MAGNITUDE']

def missing_swap(df, col1, col2):
    df.loc[~df[col1].isnull() & df[col2].isnull(), col2] = df.loc[~df[col1].isnull() & df[col2].isnull(), col1]
    df.loc[df[col1].isnull() & ~df[col2].isnull(), col1] = df.loc[df[col1].isnull() & ~df[col2].isnull(), col2]
    return df

In [5]:
# Damage variables cleaning

NOAA_df['DAMAGE_PROPERTY'] = NOAA_df.DAMAGE_PROPERTY.map(replace_str2num)

NOAA_df['DAMAGE_CROPS'] = NOAA_df.DAMAGE_CROPS.map(replace_str2num)

In [6]:
# Using data after Year 2005 and checking how many columns have NULL values

NOAA_df = NOAA_df[NOAA_df['YEAR']> 1950]

for i in NOAA_df.columns:
    if NOAA_df[i].isna().sum() > 0:
        print(NOAA_df[i].name, ":", NOAA_df[i].isna().sum(), "\n")
        print((NOAA_df[i].isna().sum()/NOAA_df.shape[0])*100, "\n")
    else:
        continue

EPISODE_ID : 232016 

13.568798127167131 

STATE : 1 

5.848216557119823e-05 

STATE_FIPS : 1 

5.848216557119823e-05 

CZ_NAME : 1557 

0.09105673179435565 

WFO : 125350 

7.330739454349698 

DAMAGE_PROPERTY : 544204 

31.826228432508362 

DAMAGE_CROPS : 655519 

38.33617069306629 

SOURCE : 345642 

20.2138926723601 

MAGNITUDE : 716476 

41.90106805978983 

MAGNITUDE_TYPE : 1284042 

75.09355684437253 

FLOOD_CAUSE : 1612631 

94.31015314724698 

CATEGORY : 1709479 

99.97403391848638 

TOR_F_SCALE : 1639007 

95.85267874635291 

TOR_LENGTH : 1442367 

84.35274570843248 

TOR_WIDTH : 1442367 

84.35274570843248 

TOR_OTHER_WFO : 1707365 

99.85040262046887 

TOR_OTHER_CZ_STATE : 1707365 

99.85040262046887 

TOR_OTHER_CZ_FIPS : 1707365 

99.85040262046887 

TOR_OTHER_CZ_NAME : 1707365 

99.85040262046887 

BEGIN_RANGE : 766318 

44.81593615618949 

BEGIN_AZIMUTH : 984815 

57.59411388699959 

BEGIN_LOCATION : 733907 

42.92047068786138 

END_RANGE : 766695 

44.83798393260983 

END

In [7]:
# Removing Outliers

NOAA_df.shape

Q1 = NOAA_df.quantile(0.25)
Q3 = NOAA_df.quantile(0.75)
IQR = Q3 - Q1

NOAA_df = NOAA_df[~((NOAA_df < (Q1 - 1.5 * IQR)) |(NOAA_df > (Q3 + 1.5 * IQR))).any(axis=1)]

NOAA_df.shape

(1198894, 51)

In [8]:
# Removing inconsistencies
NOAA_df = missing_swap(NOAA_df, 'BEGIN_RANGE', 'END_RANGE')
NOAA_df = missing_swap(NOAA_df, 'BEGIN_LAT', 'END_LAT')
NOAA_df = missing_swap(NOAA_df, 'BEGIN_LON', 'END_LON')
NOAA_df = missing_swap(NOAA_df, 'BEGIN_AZIMUTH', 'END_AZIMUTH')
NOAA_df = missing_swap(NOAA_df, 'BEGIN_LOCATION', 'END_LOCATION')

In [9]:
# calculating distance w.r.t Latitude and Longitude

def geo_distance(x):
    # Source : https://en.wikipedia.org/wiki/Haversine_formula
    p = pi/180
    lat1 = x['BEGIN_LAT']
    lat2 = x['END_LAT']
    lon1 = x['BEGIN_LON']
    lon2 = x['END_LON']
    R = 6371
    dLat = p * (lat2-lat1)
    dLon = p * (lon2-lon1)
    a = sin(dLat/2) * 2 + cos(p*lat1) * cos(p*lat2) * sin(dLon/2) * 2
    if a < 0 :
      return a
    else:
      return 2 * R * atan2(sqrt(a), sqrt(1-a))

NOAA_df['GEO_DISTANCE'] = NOAA_df.apply(lambda x: geo_distance(x), axis=1)

In [10]:
# NOAA_df['DURATION_OF_STORM'] = (pd.to_datetime(NOAA_df['END_DATE_TIME']) - pd.to_datetime(NOAA_df['BEGIN_DATE_TIME']))
# NOAA_df['DURATION_OF_STORM'] = NOAA_df['DURATION_OF_STORM'].astype('str').str[:-13]

# calculate duration of the storm event

def calc_duration(x):
    begin_dt = dt.strptime(x['BEGIN_DATE_TIME'], "%d-%b-%y %H:%M:%S")
    end_dt = dt.strptime(x['END_DATE_TIME'], "%d-%b-%y %H:%M:%S")
    difference = end_dt - begin_dt
    difference_days = difference.days + difference.seconds/86400
    return difference_days

NOAA_df['DURATION_OF_STORM'] = NOAA_df.apply(lambda x: calc_duration(x), axis=1)

In [11]:
# Dropping NAN values - if required

NOAA_df.dropna(subset=["DAMAGE_PROPERTY"], inplace=True)
# NOAA_df.dropna(subset=["DAMAGE_CROPS"], inplace=True)

In [12]:
# impute DAMAGE_PROPERTY and DAMAGE_CROPS with mean value of damage per EVENT_TYPE

X = NOAA_df.groupby('EVENT_TYPE')

Y = X['DAMAGE_CROPS'].mean()

reset_index = Y.reset_index()

# Y1 = X['DAMAGE_PROPERTY'].mean()

# reset_index1 = Y1.reset_index()

NOAA_df = pd.merge(NOAA_df, reset_index, on='EVENT_TYPE', how='inner')

# NOAA_df = pd.merge(NOAA_df, reset_index1, on='EVENT_TYPE', how='inner')

# NOAA_df.drop(columns=['DAMAGE_CROPS_x','DAMAGE_PROPERTY_x'], inplace=True)

# NOAA_df = NOAA_df.rename(columns={"DAMAGE_CROPS_y": "DAMAGE_CROPS", "DAMAGE_PROPERTY_y":"DAMAGE_PROPERTY"})

NOAA_df.drop(columns=['DAMAGE_CROPS_x'], inplace=True)

NOAA_df = NOAA_df.rename(columns={"DAMAGE_CROPS_y": "DAMAGE_CROPS"})

In [13]:
# Update EVENT_TYPE column

NOAA_df.replace({'EVENT_TYPE' : { 'THUNDERSTORMWIND/TREE' : 'ThunderstormWind', 'THUNDERSTORMWIND/TREES' : 'ThunderstormWind', 'THUNDERSTORMWINDS/FLASHFLOOD' : 'ThunderstormWind'
                                 , 'THUNDERSTORMWINDS/FLOODING' : 'ThunderstormWind', 'THUNDERSTORMWINDS/HEAVYRAIN' : 'ThunderstormWind'
                                 , 'THUNDERSTORMWINDSFUNNELCLOU': 'ThunderstormWind', 'THUNDERSTORMWINDSHEAVYRAIN': 'ThunderstormWind'
                                 , 'THUNDERSTORMWINDSLIGHTNING': 'ThunderstormWind', 'HAIL/ICYROADS': 'Hail', 'HAILFLOODING': 'Hail'}})

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,GEO_DISTANCE,DURATION_OF_STORM,DAMAGE_CROPS
0,197908,8,2000,197908,8,2000,NaN,9986735,COLORADO,8.0,...,39.98,-103.58,39.98,-103.58,NaN,NaN,PUB,0.0,0.000000,0.0
1,197908,30,1650,197908,30,1650,NaN,9986744,COLORADO,8.0,...,40.58,-105.08,40.58,-105.08,NaN,NaN,PUB,0.0,0.000000,0.0
2,197908,30,1700,197908,30,1700,NaN,9986745,COLORADO,8.0,...,39.73,-104.85,39.73,-104.85,NaN,NaN,PUB,0.0,0.000000,0.0
3,197908,1,2000,197908,1,2000,NaN,9986733,COLORADO,8.0,...,40.33,-103.80,40.33,-103.80,NaN,NaN,PUB,0.0,0.000000,0.0
4,197908,8,1900,197908,8,1900,NaN,9986734,COLORADO,8.0,...,40.63,-103.20,40.63,-103.20,NaN,NaN,PUB,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742132,201806,15,1001,201806,16,1015,126097.0,755819,HAWAII,15.0,...,NaN,NaN,NaN,NaN,Halemaumau Crater on the Kilauea Volcano Summi...,NaN,CSV,NaN,1.009722,0.0
742133,201806,17,318,201806,18,1000,126097.0,755844,HAWAII,15.0,...,NaN,NaN,NaN,NaN,Halemaumau Crater on the Kilauea Volcano Summi...,NaN,CSV,NaN,1.279167,0.0
742134,201806,17,318,201806,18,1000,126097.0,755845,HAWAII,15.0,...,NaN,NaN,NaN,NaN,Halemaumau Crater on the Kilauea Volcano Summi...,NaN,CSV,NaN,1.279167,0.0
742135,201908,11,1630,201908,15,1000,141305.0,848662,IDAHO,16.0,...,NaN,NaN,NaN,NaN,The Desert Ride Fire about 12 miles west of Re...,The Desert Ride Fire about 12 miles west of Re...,CSV,NaN,3.729167,0.0


In [14]:
def impute_NOAA_data(df):
    
    # magnitude converted into other variables
    drop_list = ['EVENT_NARRATIVE', 'EPISODE_NARRATIVE', 'EPISODE_ID', 'MAGNITUDE', 'BEGIN_LAT', 'END_LAT', 'BEGIN_LON', 'END_LON','BEGIN_DATE_TIME','END_DATE_TIME','STATE_FIPS', 'TOR_OTHER_CZ_FIPS', 'WFO', 'SOURCE'
               , 'CATEGORY', 'CZ_FIPS','DATA_SOURCE', 'TOR_OTHER_WFO', 'EVENT_ID']
    #  impute_mean_list = ['BEGIN_LAT', 'END_LAT', 'BEGIN_LON', 'END_LON'] # removed because for events like DROUGHT there should be no travelled distance and added to drop list
    # Instead impute GEO_DISTANCE to 0
    
    # imputing damage columns with 0 for the time-being
    impute_zero_list = ['BEGIN_RANGE', 'END_RANGE', 'WIND_SPEED', 'HAIL_SIZE','GEO_DISTANCE']
    impute_NA_list = ['CZ_NAME', 'STATE', 'MAGNITUDE_TYPE', 'BEGIN_AZIMUTH', 'END_AZIMUTH', 'BEGIN_LOCATION', 'END_LOCATION', 'FLOOD_CAUSE', 'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH','TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_NAME']
      
    # Splitting magnitude variable into constituent attributes
    df['WIND_SPEED'] = df.apply(winds, axis = 1)
    df['HAIL_SIZE'] = df.apply(hail, axis = 1)
    
    # Imputing string columns with missing values with NA
    for col in impute_NA_list:
        df[col] = df[col].astype('str').apply(lambda x: 'NA' if x=='nan' else x) # changed from N/A to NA
        
    # Imputing float columns having missing values with 0.0
    for col in impute_zero_list:
        df[col] = df[col].fillna(0.0)
        
    # Imputing latitude and longitudes with average value
    # for col in impute_mean_list:
    #     df[col] = df[col].fillna(np.mean)
        
    # Dropping text and ID columns
    for col in drop_list:
        df.drop(col, axis=1, inplace=True)
    
    return df

imputed_NOAA_df = impute_NOAA_data(NOAA_df.copy())
imputed_NOAA_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,STATE,YEAR,MONTH_NAME,EVENT_TYPE,...,BEGIN_AZIMUTH,BEGIN_LOCATION,END_RANGE,END_AZIMUTH,END_LOCATION,GEO_DISTANCE,DURATION_OF_STORM,DAMAGE_CROPS,WIND_SPEED,HAIL_SIZE
0,197908,8,2000,197908,8,2000,COLORADO,1979,August,Thunderstorm Wind,...,NA,NA,0.0,NA,NA,0.0,0.0,0.0,0.0,56.0
1,197908,30,1650,197908,30,1650,COLORADO,1979,August,Thunderstorm Wind,...,NA,NA,0.0,NA,NA,0.0,0.0,0.0,0.0,50.0
2,197908,30,1700,197908,30,1700,COLORADO,1979,August,Thunderstorm Wind,...,NA,NA,0.0,NA,NA,0.0,0.0,0.0,0.0,50.0
3,197908,1,2000,197908,1,2000,COLORADO,1979,August,Thunderstorm Wind,...,NA,NA,0.0,NA,NA,0.0,0.0,0.0,0.0,61.0
4,197908,8,1900,197908,8,1900,COLORADO,1979,August,Thunderstorm Wind,...,NA,NA,0.0,NA,NA,0.0,0.0,0.0,0.0,0.0


In [15]:
def get_EPA_data():
    
    ground_temp_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-04/temperature_fig-1.csv', skiprows=6, usecols = [0, 1], encoding='latin1')
    ground_temp_df.columns = ["YEAR", "SURFACE_TEMP_DIFF"]
    
    greenhouse_df = pd.read_csv("https://www.epa.gov/sites/default/files/2021-04/us-ghg-emissions_fig-1.csv",skiprows=6)
    greenhouse_df.columns = ["YEAR", "CO2", "METHANE", "N2O", "HFC_PFC_SF6_NF3"]
    
    precipitation_df =  pd.read_csv('https://www.epa.gov/sites/default/files/2021-04/heavy-precip_fig-1.csv',skiprows=6)
    precipitation_df.columns = ["YEAR", "PPT_LAND_AREA", "PPT_NINE_YEAR_AVG"]
    
    sea_level_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-04/sea-level_fig-1.csv', skiprows=6)
    sea_level_df.columns = ["YEAR", "CSIRO_ADJ_SEA_LEVEL", "CSIRO_LOWER", "CSIRO_UPPER", "NOAA_ADJ_SEA_LEVEL"]
    sea_level_df.loc[sea_level_df['YEAR'] > 1992, 'CSIRO_ADJ_SEA_LEVEL'] = sea_level_df.loc[sea_level_df['YEAR'] > 1992, 'NOAA_ADJ_SEA_LEVEL']
    sea_level_df.drop(['CSIRO_LOWER', 'CSIRO_UPPER', 'NOAA_ADJ_SEA_LEVEL'], axis=1, inplace=True)
    
    seasonal_temp_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-04/seasonal-temperature_fig-1.csv', skiprows=6)
    seasonal_temp_df.columns = ["YEAR", "WINTER_ANOMALY", "SPRING_ANOMALY", "SUMMER_ANOMALY", "FALL_ANOMALY"]
    
    arctic_ice_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-03/arctic-sea-ice_fig-1.csv', skiprows=6)
    arctic_ice_df.columns = ["YEAR", "ICE_CVG_MARCH", "ICE_CVG_SEP"]
    
    glacier_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-03/glaciers_fig-1.csv', skiprows=6)
    glacier_df.columns = ["YEAR", "GLACIER_MASS_BAL", "NUM_OBS"]
    glacier_df.drop(['NUM_OBS'], axis=1, inplace=True)
    
    dfs = [ground_temp_df, greenhouse_df, precipitation_df, sea_level_df, seasonal_temp_df, arctic_ice_df, glacier_df]
    epa_df = reduce(lambda left, right: pd.merge(left, right, how="outer", on="YEAR"), dfs)
    epa_df = epa_df[epa_df.YEAR >= 1950]
    
    return epa_df

epa_source_df = get_EPA_data()

In [18]:
imputed_NOAA_df.to_pickle('Data/imputed_NOAA_df.pkl')

In [19]:
imputed_NOAA_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1165719 entries, 0 to 1165718
Data columns (total 36 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   BEGIN_YEARMONTH     1165719 non-null  int64  
 1   BEGIN_DAY           1165719 non-null  int64  
 2   BEGIN_TIME          1165719 non-null  int64  
 3   END_YEARMONTH       1165719 non-null  int64  
 4   END_DAY             1165719 non-null  int64  
 5   END_TIME            1165719 non-null  int64  
 6   STATE               1165719 non-null  object 
 7   YEAR                1165719 non-null  int64  
 8   MONTH_NAME          1165719 non-null  object 
 9   EVENT_TYPE          1165719 non-null  object 
 10  CZ_TYPE             1165719 non-null  object 
 11  CZ_NAME             1165719 non-null  object 
 12  CZ_TIMEZONE         1165719 non-null  object 
 13  INJURIES_DIRECT     1165719 non-null  int64  
 14  INJURIES_INDIRECT   1165719 non-null  int64  
 15  DEATHS_DIRECT  

In [ ]:
def impute_EPA_DATA(df, breaks):
    fillable_cols = df.columns[df.isnull().sum() > 0]
    for col in fillable_cols:
        temp_df = df[['YEAR', col]]
        present_df = temp_df[~ temp_df[col].isnull()]
        null_df = temp_df[temp_df[col].isnull()]
        years = sorted(np.random.choice(present_df['YEAR'], breaks))
        input_df = present_df[present_df['YEAR'].isin(years)]
        func = scipy.interpolate.interp1d(input_df['YEAR'], input_df[col], fill_value="extrapolate")
        temp_df['INTERPOLATION'] = func(temp_df['YEAR'])
        df[col] = temp_df.apply(lambda x: x['INTERPOLATION'] if isnan(x[col]) else x[col], axis=1)
    return df
        
imputed_EPA_df = impute_EPA_DATA(epa_source_df.copy(), 6)
imputed_EPA_df.head()

In [53]:
#cleaned_df = pd.merge(imputed_NOAA_df, imputed_EPA_df, on='YEAR', how='inner')
cleaned_df = imputed_NOAA_df
cleaned_df.to_pickle('/content/drive/MyDrive/cleaned.pkl')

In [ ]:
import pandas as pd

NOAA = pd.read_pickle('/content/drive/MyDrive/cleaned.pkl')

df_train = NOAA[NOAA['YEAR'] >= 1950]

In [ ]:
# Normalizing DAMAGE_PROPERTY variable - if required

# df_train['HasDamage'] = pd.Series(len(df_train['DAMAGE_PROPERTY']), index=df_train.index)
# df_train['HasDamage'] = 0 
# df_train.loc[df_train['DAMAGE_PROPERTY']>0,'HasDamage'] = 1

# df_train.loc[df_train['HasDamage']==1,'DAMAGE_PROPERTY'] = np.log(df_train['DAMAGE_PROPERTY'])

# df_train.drop(columns=['HasDamage'], inplace=True)

# df_train.info()

In [5]:
# assign labels to categorical columns

# def mapping(xx):
#     dict = {}
#     count = -1
#     for x in xx:
#         dict[x] = count + 1
#         count = count + 1
#     return dict

# for i in ['STATE', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE',  'MAGNITUDE_TYPE', 'BEGIN_AZIMUTH', 'END_AZIMUTH', 'FLOOD_CAUSE', 'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_NAME'
#           , 'CZ_NAME',  'TOR_OTHER_CZ_NAME', 'BEGIN_LOCATION', 'END_LOCATION','CZ_TIMEZONE']:
#     unique_tag = df_train[i].value_counts().keys().values
#     dict_mapping = mapping(unique_tag)
#     df_train[i] = df_train[i].map(lambda x: dict_mapping[x] if x in dict_mapping.keys() else -1)


In [55]:
# GET DUMMIES and assign labels to categorical columns

def mapping(xx):
    dict = {}
    count = -1
    for x in xx:
        dict[x] = count + 1
        count = count + 1
    return dict

for i in ['CZ_NAME', 'BEGIN_LOCATION', 'END_LOCATION', 'TOR_LENGTH' ,'TOR_WIDTH',	'TOR_OTHER_CZ_STATE',	'TOR_OTHER_CZ_NAME']:
    unique_tag = df_train[i].value_counts().keys().values
    dict_mapping = mapping(unique_tag)
    df_train[i] = df_train[i].map(lambda x: dict_mapping[x] if x in dict_mapping.keys() else -1)

df_train = pd.get_dummies(df_train, prefix=['STATE', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_TIMEZONE', 'BEGIN_AZIMUTH', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'TOR_F_SCALE', 'END_AZIMUTH']
                          , columns=['STATE', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_TIMEZONE','BEGIN_AZIMUTH', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'TOR_F_SCALE', 'END_AZIMUTH'])
df_train.head()


In [31]:
# remove any pending NAN from DAMAGE_CROPS and DAMAGE_PROPERTY

# df_train.dropna(subset=["DAMAGE_PROPERTY"], inplace=True)
# df_train.dropna(subset=["DAMAGE_CROPS"], inplace=True)

# check for NULL values
for i in df_train.columns:
    if df_train[i].isna().sum() > 0:
        print(df_train[i].name, ":", df_train[i].isna().sum(), "\n")
        print((df_train[i].isna().sum()/df_train.shape[0])*100, "\n")
    else:
        continue

In [ ]:
# correlation

corrmat = df_train.corr()

f, ax = plt.subplots(figsize=(22, 19))
sns.heatmap(corrmat, vmax=1.0, square=True)
plt.show()

#saleprice correlation matrix
# k = 20 #number of variables for heatmap
# cols = corrmat.nlargest(k, 'DAMAGE_PROPERTY')['DAMAGE_PROPERTY'].index
# cm = np.corrcoef(df_train[cols].values.T)
# sns.set(font_scale=1.25)
# hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 15}, yticklabels=cols.values, xticklabels=cols.values)
# plt.show()

In [ ]:
# Create train test split
X = df_train.loc[:, ~df_train.columns.isin(['DAMAGE_PROPERTY'])]

Y = df_train['DAMAGE_PROPERTY']

# split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

print(X_train)

In [57]:
# Scaling the data if required

# sc = StandardScaler()
# sc.fit(X_train)
# X_train = sc.transform(X_train)
# X_test = sc.transform(X_test)


# Normalize the data

X_train = (X_train - X_train.mean()) / X_train.std()

In [37]:
# PCA code if we use one-hot encoding

# pca = PCA(0.95)
# #pca = PCA(n_components=25)
# pca.fit(X_train)


# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

# print(X_train.shape)
# pca.components_ 

In [ ]:
print(X_train.shape)


In [60]:
model = LinearRegression()
model.fit(X_train, y_train)

print(model)

# make predictions for test data
y_pred = model.predict(X_test)
y_pred_score = [round(value) for value in y_pred]

MSE = mean_squared_error(y_pred,y_test)
print(MSE)

print(model.score(X_train,y_train))

print(r2_score(y_test, y_pred))


LinearRegression()
1227284.1892350877
0.30910453397811843
0.3056530742410414


In [40]:
clf_rf = RandomForestRegressor(n_estimators= 100, oob_score = 'TRUE', n_jobs = -1,random_state =50, max_features = "auto", min_samples_leaf = 50)
# RandomForestRegressor(max_depth=10, random_state=0)

# perform training
clf_rf.fit(X_train, y_train)

# make predictions

# prediction on test using all features
y_pred = clf_rf.predict(X_test)

MSE = mean_squared_error(y_pred,y_test)
print(MSE)

print(clf_rf.score(X_train,y_train))

print(r2_score(y_test, y_pred))

864152.8230748259
0.5487856014470041
0.5110978684880656


In [41]:
# np.argsort(clf_rf.feature_importances_)[::-1]
df_train.columns[np.argsort(clf_rf.feature_importances_)[::-1]]

Index(['MAGNITUDE_TYPE_E', 'CZ_TIMEZONE_EDT', 'EVENT_ID', 'CZ_TIMEZONE_CDT-5',
       'EVENT_TYPE_HAIL/ICY ROADS', 'GEO_DISTANCE', 'EVENT_TYPE_Landslide',
       'CZ_NAME', 'DAMAGE_CROPS', 'END_YEARMONTH',
       ...
       'STATE_GEORGIA', 'EVENT_TYPE_THUNDERSTORM WINDS/FLOODING',
       'EVENT_TYPE_Thunderstorm Wind', 'STATE_DELAWARE',
       'EVENT_TYPE_Tropical Storm', 'EVENT_TYPE_Tsunami', 'CZ_TIMEZONE_AST',
       'CZ_TIMEZONE_CDT', 'EVENT_TYPE_Dense Fog', 'END_AZIMUTH_WSW'],
      dtype='object', length=228)

In [ ]:
# rfc = XGBRegressor()

# parameters = {
#     'max_depth': range(2, 10, 1),
#     'n_estimators': range(60, 220, 40),
#     'learning_rate': [0.1, 0.01, 0.05]
# }

# xgb = GridSearchCV(rfc, parameters, cv=5, scoring='accuracy')

xgb = XGBRegressor(learning_rate =0.01,subsample =0.7, max_depth=5, n_estimators=50, colsample_bytree=0.8)
xgb.fit(X_train, y_train, eval_set=[(X_train, y_train)])

# print(xgb.feature_importances_)

# make predictions for test data
y_pred = xgb.predict(X_test)
y_pred_score = [round(value) for value in y_pred]

MSE = mean_squared_error(y_pred,y_test)
print(MSE)

print(xgb.score(X_train,y_train))

print(r2_score(y_test, y_pred))

In [ ]:
print(xgb.score(X_train,y_train))

print(r2_score(y_test, y_pred))

In [28]:
clf_dt = DecisionTreeRegressor()

# perform training
clf_dt.fit(X_train, y_train)

# make predictions

# prediction on test using all features
y_pred = clf_dt.predict(X_test)

MSE = mean_squared_error(y_pred,y_test)
print(MSE)

print(clf_dt.score(X_train,y_train))

print(r2_score(y_test, y_pred))

1662544.6077184037
1.0
0.22235885366291253


In [27]:
alpha_range = 10.**np.arange(-2, 3)

ridgeregcv = RidgeCV(alphas=alpha_range, normalize=True, scoring='neg_mean_squared_error')
ridgeregcv.fit(X_train, y_train)
ridgeregcv.alpha_

y_pred = ridgeregcv.predict(X_test)

print("R-Square Value",r2_score(y_test,y_pred))

R-Square Value 0.31292961784566276


In [29]:
lassoregcv = LassoCV(n_alphas=100, normalize=True, random_state=1)
lassoregcv.fit(X_train, y_train)
print('alpha : ',lassoregcv.alpha_)

y_pred = lassoregcv.predict(X_test)

print("R-Square Value",r2_score(y_test,y_pred))

alpha :  0.0007950231537356949
R-Square Value 0.3132390480735068


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

model = XGBRegressor()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1,verbose=3)
# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

In [ ]:
NOAA_df['null_cnt_damage_property'] = NOAA_df['DAMAGE_CROPS'].isnull()
df1 = NOAA_df.groupby('YEAR').agg({'null_cnt_damage_property' : ['sum', 'count']})
df1.plot()

In [ ]:
NOAA_df['null_cnt_damage_property'] = NOAA_df['DAMAGE_PROPERTY'].isnull()
df1 = NOAA_df.groupby('EVENT_TYPE').agg({'null_cnt_damage_property' : ['sum', 'count']}).reset_index()
df1['perc'] = df1[('null_cnt_damage_property', 'sum')] / df1[('null_cnt_damage_property', 'count')] * 100
plt.figure(figsize=(20, 20))
# sns.barplot(data=df1, y='EVENT_TYPE', x='perc')
sns.barplot(data=df1, y='EVENT_TYPE', x=('null_cnt_damage_property', 'sum'))

plt.show()

In [ ]:
df2 = NOAA_df.groupby('EVENT_TYPE').mean()['DAMAGE_PROPERTY'].reset_index()
df2 = df2[df2.EVENT_TYPE != 'TORNADOES, TSTM WIND, HAIL']
plt.figure(figsize=(20, 20))
# df2.head()
# sns.barplot(data=df1, y='EVENT_TYPE', x='perc')
sns.barplot(data=df2, y='EVENT_TYPE', x='DAMAGE_PROPERTY')
plt.show()
# NOAA_df['DAMAGE_PROPERTY'] = NOAA_df.DAMAGE_PROPERTY.map(replace_str2num)